In [8]:
# %pip install deepchem

In [34]:
#code from the research paper
# https://github.com/danielvik/arc_rtpred/blob/main/notebooks_and_code/functions/featurizers.py
import pandas as pd
import tensorflow
import deepchem as dc
import os
import subprocess
from rdkit import Chem


######*
######* FUNCTIONS FOR FEATURIZING
######*

####################################
# * Get METLIN SMRT data from remote URL
####################################


def get_METLIN_data(sampling=5000, only_retained=True):
    """Function to pull METLIN SMRT data from remote URL and save subset as a csv file"""
    # * pulling data from a URL
    url = "https://figshare.com/ndownloader/files/18130628"

    # * read the data into a pandas dataframe and convert inchi to SMILES
    df = pd.read_csv(url, sep=";").rename(columns={"pubchem": "id"})
    df["mol"] = [Chem.MolFromInchi(x) for x in df["inchi"]]
    df["smiles"] = [
        Chem.MolToSmiles(mol) if mol is not None else None for mol in df["mol"]
    ]
    df = df.dropna(
        axis=0
    )  # * dropping rows with NaN values as some mols did not convert to SMILES

    # * removing non-retained compounds
    if only_retained:
        df = df[df["rt"] > 200]

    # * sampling the data
    df_subset = df[["id", "smiles", "rt"]]

    # * outputting the data in a csv file
    output_dir = "../data/metlin_smrt"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_path = os.path.join(output_dir, f"sample_dataset_{sampling}_datapoints.csv")
    df_subset.to_csv(output_path, index=False)
    print("saved to: ", output_path, "\n")
    print(df_subset.head(3))

    return output_path


####################################
# * ChemAxon-based LogD calculations
####################################


def LogD_calculations(data, path_to_chemaxon_licence=None, path_to_chemaxon_tools=None):
    """
    Function to calculate LogD using ChemAxon's commandline tool. The function takes a pandas dataframe with a column named 'smiles' and returns a dataframe with the calculated LogD values.

    """

    ##* these are hardcoded variables for property calculcations
    os.environ["CHEMAXON_LICENSE_URL"] = path_to_chemaxon_licence

    exe_file_path = os.path.join(
        path_to_chemaxon_tools, "ChemAxon/MarvinSuite/bin/cxcalc"
    )
    input_file_path = os.path.join(
        path_to_chemaxon_tools, "smiles_for_chemaxon_logd_calc.smiles"
    )
    output_file_path = os.path.join(path_to_chemaxon_tools, "ChemAxon_LogD_out.csv")

    ##* saving data for the commandline tool to access it
    data.to_csv(input_file_path, index=False, header=False)

    ##* running the commandline tool
    filepath = f'{exe_file_path} -g logd -m weighted -H 7.4 logd -H 7 logd -H 6.5 logd -H 6 logd -H 5.5 logd -H 5 logd -H 4.5 logd -H 4 logd -H 3.5 logd -H 3 logd -H 2.5 logd -H 2 logd -H 1.5 logd -H 1 logd -H 0.5 logd -H 0 "{input_file_path}" > "{output_file_path}"'
    subprocess.run(filepath, shell=True)

    ##* reading the output file
    logd_desc = pd.read_csv(output_file_path, sep="\t")
    # drop rows with 'logd:FAILED' values
    logd_desc = logd_desc[~logd_desc["logD[pH=7.4]"].str.contains("FAILED")]
    # drop rows with NA values
    logd_desc = logd_desc.dropna(axis=0)

    logd_desc = logd_desc.replace(",", ".", regex=True).apply(pd.to_numeric)
    logd_desc = pd.concat([data, logd_desc], axis=1).drop(["id"], axis=1)

    ##* output the descriptors
    return logd_desc


####################################
# * Get all features
####################################


def get_features(
    path_to_data,
    feature_dir,
    feature_list,
    path_to_chemaxon_licence=None,
    path_to_chemaxon_tools=None,
):
    """function to get the features from the input feature_list. Features are saved to a csv file in the feature_dir"""

    # ECFP4 arguments
    nBits = 2048
    radius = 2

    data = pd.read_csv(path_to_data)

    ###* LOG D Calculations
    if "logD" in feature_list and path_to_chemaxon_licence is not None:
        target_dir = os.path.join(feature_dir, "logd_calculations")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        ##* the SMILES must be saved in a flat file for the cmdline tool to read them
        # subsetting the test data, to save time
        data_for_calc = data[["smiles"]].reset_index(drop=True)

        print("ChemAxon-based LogD calculations - to .CSV")

        desc = LogD_calculations(
            data_for_calc,
            path_to_chemaxon_licence=path_to_chemaxon_licence,
            path_to_chemaxon_tools=path_to_chemaxon_tools,
        )

        desc = data.merge(desc, on="smiles", how="left")

        desc.to_csv(os.path.join(target_dir, "all_data.csv"), index=False)

        NA_alert = desc.isna().any().any()
        if NA_alert:
            print("*** NA values in:", target_dir)

    ###* ECFP4 Calculations
    if "ecfp4" in feature_list:
        target_dir = os.path.join(feature_dir, "ecfp4_disk")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print("ECFP4 Featurization - to DiskDataset")

        ## creating a deepchem dataset with featurized smiles
        featurizer = dc.feat.CircularFingerprint(size=nBits, radius=radius)
        feats = featurizer.featurize(data.smiles)
        dataset = dc.data.DiskDataset.from_numpy(
            feats,
            data.rt,
            tasks=["RT_sec"],
            ids=data.smiles,
            data_dir=os.path.join(target_dir, "all_data"),
        )

        target_dir = os.path.join(feature_dir, "ecfp4_csv")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print("ECFP4 Featurization - to .CSV")
        feats_df = dataset.to_dataframe()
        feats_df.to_csv(os.path.join(target_dir, "all_data.csv"), index=False)

        NA_alert = feats_df.isna().any().any()
        if NA_alert:
            print("*** NA values in:", target_dir)

    ###* RDKit Calculations
    if "rdkit" in feature_list:
        target_dir = os.path.join(feature_dir, "rdkit_disk")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print("RDKit Descriptors - to diskdataset")
        ## creating a deepchem dataset with featurized smiles
        featurizer = dc.feat.RDKitDescriptors(is_normalized=True, use_bcut2d=False)
        feats = featurizer.featurize(data.smiles)
        dataset = dc.data.DiskDataset.from_numpy(
            feats,
            data.rt,
            tasks=["RT_sec"],
            ids=data.smiles,
            data_dir=os.path.join(target_dir, "all_data"),
        )

        target_dir = os.path.join(feature_dir, "rdkit_csv")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print("RDKit Descriptors - to .CSV")
        feats_df = dataset.to_dataframe()
        feats_df = feats_df.fillna(0)

        feats_df.to_csv(os.path.join(target_dir, "all_data.csv"), index=False)

        NA_alert = feats_df.isna().any().any()
        if NA_alert:
            print("*** NA values in:", target_dir)

    ###* MolGraphConv
    if "molgraphconv" in feature_list:
        target_dir = os.path.join(feature_dir, "molgraphconv")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print("MolGraphConv Feat - to diskdataset")
        ## creating a deepchem dataset with featurized smiles
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        feats = featurizer.featurize(data.smiles)
        dataset = dc.data.DiskDataset.from_numpy(
            feats,
            data.rt,
            tasks=["RT_sec"],
            ids=data.smiles,
            data_dir=os.path.join(target_dir, "all_data"),
        )

In [35]:
#data splitting
import pandas as pd
import numpy as np
import deepchem as dc
import os
import re


def data_splitter(path_to_data, destination_dir, number_of_splits=5, seed=42):
    ##* Loading data into diskDataset and using scaffold-split
    data = pd.read_csv(path_to_data)

    dataset = dc.data.NumpyDataset(X=data.id, y=data.rt, ids=data.smiles)

    splitter = dc.splits.ScaffoldSplitter()

    train_dataset, test_dataset = splitter.train_test_split(
        dataset=dataset, frac_train=0.9, seed=seed
    )

    ##* saving test and train splits as .CSV
    train_df = train_dataset.to_dataframe()
    train_df = train_df.rename(columns={"X": "id", "y": "rt", "ids": "smiles"})
    train_df.to_csv(os.path.join(destination_dir, "train_df.csv"), index=False)

    test_df = test_dataset.to_dataframe()
    test_df = test_df.rename(columns={"X": "id", "y": "rt", "ids": "smiles"})
    test_df.to_csv(os.path.join(destination_dir, "test_df.csv"), index=False)

    ##* Preparing Cross validation splits
    target_dir = os.path.join(destination_dir, "cv_splits")
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    cv_splits = splitter.k_fold_split(dataset=train_dataset, k=number_of_splits)

    for i, k_split in enumerate(cv_splits):
        train_k_split = k_split[0]
        train_k_split = train_k_split.to_dataframe()
        train_k_split = train_k_split.rename(
            columns={"X": "id", "y": "rt", "ids": "smiles"}
        )
        train_k_split.to_csv(
            os.path.join(target_dir, f"train_{i}_split.csv"), index=False
        )

        valid_k_split = k_split[1]
        valid_k_split = valid_k_split.to_dataframe()
        valid_k_split = valid_k_split.rename(
            columns={"X": "id", "y": "rt", "ids": "smiles"}
        )
        valid_k_split.to_csv(
            os.path.join(target_dir, f"valid_{i}_split.csv"), index=False
        )


def feature_splitter_csv(
    path_to_features, path_to_splits, save_folder_name="data_splits"
):
    ##*  collecting the needed paths for the features

    csv_feature_paths = []
    csv_feature_subdirs = []

    for subdir, dirs, files in os.walk(path_to_features):
        for file in files:
            # csv-based features
            if file.startswith("all_data") and file.endswith(".csv"):
                csv_feature_paths.append(os.path.join(subdir, file))
                csv_feature_subdirs.append(subdir)

    ##* getting the paths for the premade-splits

    split_paths = []
    output_paths = []

    for subdir, dirs, files in os.walk(path_to_splits):
        for file in files:
            if file.endswith(".csv"):
                split_paths.append(os.path.join(subdir, file))
                output_paths.append(file)

    ##* walking through each .CSV feature dir

    for i, path in enumerate(csv_feature_paths):
        features = pd.read_csv(path).rename(columns={"y": "rt", "ids": "id"})

        # two types of feature output: Deepchem generated
        if "w" in features:
            features = features.drop(["w", "rt"], axis=1)

        # or Chemaxon (cxcalc) generated
        elif "logD[pH=4.5]" in features:
            features = features.drop(["smiles", "rt"], axis=1)

        ##* for each feature type, the features are split according to the premade_splits
        for j, split in enumerate(split_paths):
            split = pd.read_csv(split_paths[j])
            merged_split = pd.concat([split, features], axis=1, join="inner")

            save_dir = os.path.join(csv_feature_subdirs[i], save_folder_name)  #!
            if not os.path.exists(save_dir):  #!
                os.makedirs(save_dir)  #!

            labels_split = merged_split[["smiles", "rt"]]
            labels_split.to_csv(
                os.path.join(save_dir, ("labels_" + output_paths[j])), index=False
            )

            feature_split = merged_split.drop(["id", "rt", "smiles"], axis=1)
            if "w" in feature_split:
                feature_split = feature_split.drop(["w"], axis=1)
            feature_split.to_csv(
                os.path.join(save_dir, ("features_" + output_paths[j])), index=False
            )


def feature_splitter_diskdatasets(
    path_to_features, path_to_splits, save_folder_name="data_splits"
):
    ##*  collecting the needed paths for the features

    disk_feature_subdirs = []

    for subdir, dirs, files in os.walk(path_to_features):
        for file in files:
            # csv-based features
            if file.endswith(".gzip") and ("all_data" in subdir):
                disk_feature_subdirs.append(subdir)

    ##* getting the paths for the premade-splits

    split_paths = []
    output_paths = []

    for subdir, dirs, files in os.walk(path_to_splits):
        for file in files:
            if file.endswith(".csv"):
                split_paths.append(os.path.join(subdir, file))
                output_paths.append(file)

    for i, path in enumerate(disk_feature_subdirs):
        path = os.path.abspath(path)
        ##* loading all datapoints for the specific feature set
        dataset = dc.data.DiskDataset(data_dir=path)
        array = dataset.ids

        ##* for each feature type, the features are split according to the premade_splits
        for j, split in enumerate(split_paths):
            # loading split dataset and extracts IDs
            split = pd.read_csv(split_paths[j])
            df_ids = split["smiles"].values

            # matches with the array to find indexes of datapoints present in current split
            matches = np.where(np.isin(array, df_ids))[0]

            # defining a folder name based on split name
            split_name = re.search(r"(.*).csv$", output_paths[j])[1]
            feat_dir = re.search(r"features\\(.*)\\all_data", path)[1]
            output_dir = os.path.join(
                path_to_features, feat_dir, save_folder_name, split_name
            )
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            # using the deepchem .select() function to subset the dataset according to matches
            dataset.select(matches, select_dir=output_dir)

In [36]:
import os

In [37]:
path_to_data = get_METLIN_data(sampling = 5000, only_retained = True)

[23:21:49] Explicit valence for atom # 14 N, 4, is greater than permitted
[23:21:49] ERROR: Explicit valence for atom # 14 N, 4, is greater than permitted

[23:21:49] Explicit valence for atom # 16 N, 4, is greater than permitted
[23:21:49] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted

[23:21:49] Explicit valence for atom # 16 N, 4, is greater than permitted
[23:21:49] ERROR: Explicit valence for atom # 16 N, 4, is greater than permitted

[23:21:49] Explicit valence for atom # 18 N, 4, is greater than permitted
[23:21:49] ERROR: Explicit valence for atom # 18 N, 4, is greater than permitted

[23:21:50] Explicit valence for atom # 17 N, 4, is greater than permitted
[23:21:50] ERROR: Explicit valence for atom # 17 N, 4, is greater than permitted

[23:21:50] Explicit valence for atom # 15 N, 4, is greater than permitted
[23:21:50] ERROR: Explicit valence for atom # 15 N, 4, is greater than permitted

[23:21:50] Explicit valence for atom # 16 N, 4, is greater than 

saved to:  ../data/metlin_smrt/sample_dataset_5000_datapoints.csv 

     id                                             smiles     rt
1  3505  COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...  687.8
2  2159    CCN1CCC[C@@H]1CN=C(O)c1cc(S(=O)(=O)CC)c(N)cc1OC  590.7
3  1340                                 Oc1cccc2c(O)nccc12  583.6


In [38]:
print(path_to_data)
path = "dataset.csv"

../data/metlin_smrt/sample_dataset_5000_datapoints.csv


In [39]:
path_to_features = 'features/' 
if not os.path.exists(path_to_features):
    os.makedirs(path_to_features)

In [40]:
feature_list = ['logD', 
                'ecfp4', 
                'rdkit', 
                'molgraphconv'
                ]

get_features(path, path_to_features, feature_list)
             # path_to_chemaxon_licence= path_to_chemaxon_licence, #I need licenses for this
             # path_to_chemaxon_tools = path_to_chemaxon_tools)

ECFP4 Featurization - to DiskDataset
ECFP4 Featurization - to .CSV


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


RDKit Descriptors - to diskdataset
RDKit Descriptors - to .CSV
MolGraphConv Feat - to diskdataset


In [41]:
path_to_splits = 'data_splits/'
if not os.path.exists(path_to_splits):
    os.makedirs(path_to_splits)

data_splitter(path, path_to_splits, number_of_splits = 5)

In [42]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from hyperopt import hp, fmin, tpe, Trials

def xgboost_hyperopt(
    model_check_point, X_train, y_train, X_valid, y_valid, epochs=100, iterations=20
):
    ##* writing objective function

    def objective(params):
        model = xgb.XGBRegressor(
            n_estimators=epochs,  #Number of trees (epochs)
            learning_rate=params["learning_rate"],
            max_depth=int(params["max_depth"]),
            subsample=params["subsample"],
            gamma=params["gamma"],
            colsample_bytree=params["colsample_bytree"],
            min_child_weight=int(params["min_child_weight"]),
            reg_alpha=params["reg_alpha"],
            reg_lambda=params["reg_lambda"],
            eval_metric="rmse",
            early_stopping_rounds=10
        )

        # Train the model on the training data with early stopping
        eval_set = [(X_valid, y_valid)]
        model.fit(
            X_train,
            y_train,
            eval_set=eval_set,
            verbose=False,
        )

        # Get the best iteration based on early stopping
        best_iteration = model.best_iteration

        # Score the model on the validation data using the best iteration
        y_pred = model.predict(X_valid, iteration_range = (0, best_iteration + 1))
        mse = mean_squared_error(y_valid, y_pred)
        score = mse
    
        return score

    ##* Define the search space for hyperparameters
    space = {
        "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
        "max_depth": hp.quniform("max_depth", 3, 10, 1),
        "subsample": hp.uniform("subsample", 0.7, 1.0),
        "gamma": hp.uniform("gamma", 0, 1),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.7, 1.0),
        "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1),
        "reg_alpha": hp.loguniform("reg_alpha", np.log(1e-10), np.log(1.0)),
        "reg_lambda": hp.loguniform("reg_lambda", np.log(1e-10), np.log(1.0)),
    }

    ##* running hyperopt trials
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=iterations,  # Number of iterations
        trials=trials,
    )

    ##* outputting trial results

    print("Best Parameters: {}".format(best))

    return best

def xgboost_retrain_test(
    best_params,
    X_train,
    y_train,
    X_valid,
    y_valid,
    X_test,
    y_test,
    model_check_point,
    epochs=100,
):
    # Create an XGBoost model with the best hyperparameters
    model = xgb.XGBRegressor(
        n_estimators=epochs,  # Choose an appropriate number of trees (epochs)
        learning_rate=best_params["learning_rate"],
        max_depth=int(best_params["max_depth"]),
        subsample=best_params["subsample"],
        gamma=best_params["gamma"],
        colsample_bytree=best_params["colsample_bytree"],
        min_child_weight=int(best_params["min_child_weight"]),
        reg_alpha=best_params["reg_alpha"],
        reg_lambda=best_params["reg_lambda"],
        eval_metric="rmse",
        early_stopping_rounds=10
    )

    # Train the model on the training data with early stopping
    eval_set = [(X_valid, y_valid)]
    model.fit(
        X_train,
        y_train,
        eval_set=eval_set,
        verbose=False,
    )

    # Get the best iteration based on early stopping
    best_iteration = model.best_iteration

    # ##* saving the model
    # file_name = os.path.join(model_check_point, "xgboost_model.pkl")
    # # save
    # pickle.dump(best_iteration, open(file_name, "wb"))

    # Score the model on the validation data using the best iteration
    y_pred = model.predict(X_test, iteration_range = (0, best_iteration + 1))
    y_true = y_test

    r2_metric = r2_score(y_true, y_pred)
    print(f"R2 score: {r2_metric}")

    mse_score = mean_squared_error(y_true, y_pred, squared=True)
    print(f"MSE Score: {mse_score}")

    rmse_score = mean_squared_error(y_true, y_pred, squared=False)
    print(f"RMSE Score: {rmse_score}")

    mae_score = mean_absolute_error(y_true, y_pred)
    print(f"MAE Score: {mae_score}")

    score_dict = {
        "R2 Score": r2_metric,
        "MSE Score": mse_score,
        "RMSE Score": rmse_score,
        "MAE Score": mae_score,
    }

    # with open((model_check_point + "/test_scores.json"), "w", encoding="utf-8") as f:
    #     json.dump(score_dict, f, default=int)

    df = pd.DataFrame({"actual_RT": y_true, "pred_RT": y_pred})
    # print(df.head())
    # pred_df_path = model_check_point + "/test_preds.csv"
    # df.to_csv(pred_df_path, index=False)
    # print(f"saved predictions to {pred_df_path}")

    print("\n*** Hyperparameter Optimization is done ***\n")

In [43]:
#* splitting the .CSV-based features according to the data splits
feature_splitter_csv(path_to_features, path_to_splits, save_folder_name = 'cv_splits')

#* splitting the diskDataset-based features according to the data splits
# feature_splitter_diskdatasets(path_to_features,path_to_splits, save_folder_name = 'cv_splits')

In [44]:
def run_splits():
    for i in range(5):
        print(f"Iteration {i}")
        y_train = pd.read_csv(f'features/rdkit_csv/cv_splits/labels_train_{i}_split.csv')["rt"]
        X_train = pd.read_csv(f'features/rdkit_csv/cv_splits/features_train_{i}_split.csv')
        new_column_names = ["X" + str(i) for i in range(len(X_train.columns))]
        X_train.columns = new_column_names
        
        y_valid = pd.read_csv(f'features/rdkit_csv/cv_splits/labels_valid_{i}_split.csv')["rt"]
        X_valid = pd.read_csv(f"features/rdkit_csv/cv_splits/features_valid_{i}_split.csv")
        new_column_names = ["X" + str(i) for i in range(len(X_valid.columns))]
        X_valid.columns = new_column_names
    
        y_test = pd.read_csv('features/rdkit_csv/cv_splits/labels_test_df.csv')["rt"]
        X_test = pd.read_csv('features/rdkit_csv/cv_splits/features_test_df.csv')
        new_column_names = ["X" + str(i) for i in range(len(X_test.columns))]
        X_test.columns = new_column_names

        epochs = 200
        iterations = 100
        
        best_params = xgboost_hyperopt(
            'model_check_point',
            X_train,
            y_train,
            X_valid,
            y_valid,
            epochs=epochs,
            iterations=iterations,
        )

        xgboost_retrain_test(best_params, X_train, y_train, X_valid, y_valid, X_test, y_test, "checkpoint")

In [45]:
run_splits()

Iteration 0
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.61trial/s, best loss: 33034.20281000577]
Best Parameters: {'colsample_bytree': 0.8132748059317126, 'gamma': 0.08154875888669734, 'learning_rate': 0.2395234166432407, 'max_depth': 9.0, 'min_child_weight': 1.0, 'reg_alpha': 3.212188066985545e-08, 'reg_lambda': 0.00016580822974435183, 'subsample': 0.7739758547709046}
R2 score: -0.07776937071170953
MSE Score: 25415.930323286742
RMSE Score: 159.42374454041263
MAE Score: 125.98740350341797

*** Hyperparameter Optimization is done ***

Iteration 1
  0%|                                                                                                                                                                 | 0/100 [00:00<?, ?trial/s, best loss=?]

/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.69trial/s, best loss: 28554.265390076274]
Best Parameters: {'colsample_bytree': 0.797414507459184, 'gamma': 0.20400219391862148, 'learning_rate': 0.2976297241708892, 'max_depth': 9.0, 'min_child_weight': 2.0, 'reg_alpha': 0.00022676218262877471, 'reg_lambda': 0.008386262976650318, 'subsample': 0.850528864662465}
R2 score: -0.07406525659445129
MSE Score: 25328.57999688846
RMSE Score: 159.14955229873712
MAE Score: 126.53177893066407

*** Hyperparameter Optimization is done ***

Iteration 2
  0%|                                                                                                                                                                 | 0/100 [00:00<?, ?trial/s, best loss=?]

/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:23<00:00,  4.20trial/s, best loss: 25565.974063150916]
Best Parameters: {'colsample_bytree': 0.9447058035104805, 'gamma': 0.565626587083945, 'learning_rate': 0.08332840847389344, 'max_depth': 5.0, 'min_child_weight': 1.0, 'reg_alpha': 0.5176653336202008, 'reg_lambda': 7.737759814870895e-08, 'subsample': 0.8847036289492267}
R2 score: -0.09574926082613833
MSE Score: 25839.93164192406
RMSE Score: 160.74803775450593
MAE Score: 129.07367041015627

*** Hyperparameter Optimization is done ***

Iteration 3
  0%|                                                                                                                                                                 | 0/100 [00:00<?, ?trial/s, best loss=?]

/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  3.71trial/s, best loss: 30336.734060807587]
Best Parameters: {'colsample_bytree': 0.8472500808261842, 'gamma': 0.5698275731884752, 'learning_rate': 0.14788854242963018, 'max_depth': 9.0, 'min_child_weight': 1.0, 'reg_alpha': 1.8179731438332954e-08, 'reg_lambda': 5.801003113520564e-09, 'subsample': 0.8045812962504085}
R2 score: -0.08334674289243349
MSE Score: 25547.455774449558
RMSE Score: 159.8357149527275
MAE Score: 127.56905364990234

*** Hyperparameter Optimization is done ***

Iteration 4
  0%|                                                                                                                                                                 | 0/100 [00:00<?, ?trial/s, best loss=?]

/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.47trial/s, best loss: 27773.91906430625]
Best Parameters: {'colsample_bytree': 0.9994501862770774, 'gamma': 0.6094756280429889, 'learning_rate': 0.055971223194102096, 'max_depth': 7.0, 'min_child_weight': 6.0, 'reg_alpha': 1.0988341711456445e-10, 'reg_lambda': 0.34640477612153725, 'subsample': 0.8987448049385149}
R2 score: -0.06990556805754577
MSE Score: 25230.486325929203
RMSE Score: 158.84107254085512
MAE Score: 126.49076300048827

*** Hyperparameter Optimization is done ***



/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [50]:
best_params = {'colsample_bytree': 0.9994501862770774, 'gamma': 0.6094756280429889, 'learning_rate': 0.055971223194102096, 'max_depth': 7.0, 'min_child_weight': 6.0, 'reg_alpha': 1.0988341711456445e-10, 'reg_lambda': 0.34640477612153725, 'subsample': 0.8987448049385149}
model = xgb.XGBRegressor(
        n_estimators=200,  # Choose an appropriate number of trees (epochs)
        learning_rate=best_params["learning_rate"],
        max_depth=int(best_params["max_depth"]),
        subsample=best_params["subsample"],
        gamma=best_params["gamma"],
        colsample_bytree=best_params["colsample_bytree"],
        min_child_weight=int(best_params["min_child_weight"]),
        reg_alpha=best_params["reg_alpha"],
        reg_lambda=best_params["reg_lambda"],
        eval_metric="rmse",
    )
x_train_df = pd.read_csv('features/rdkit_csv/cv_splits/features_train_df.csv')
x_test_df = pd.read_csv('features/rdkit_csv/cv_splits/features_test_df.csv')

y_train_df = pd.read_csv('features/rdkit_csv/cv_splits/labels_train_df.csv')['rt']
y_test_df = pd.read_csv('features/rdkit_csv/cv_splits/labels_test_df.csv')['rt']

model.fit(
        x_train_df,
        y_train_df,
        verbose=False,
    )

# Get the best iteration based on early stopping
# best_iteration = model.best_iteration

# ##* saving the model
# file_name = os.path.join(model_check_point, "xgboost_model.pkl")
# # save
# pickle.dump(best_iteration, open(file_name, "wb"))

# Score the model on the validation data using the best iteration
y_pred = model.predict(x_test_df)
y_true = y_test_df

r2_metric = r2_score(y_true, y_pred)
print(f"R2 score: {r2_metric}")

mse_score = mean_squared_error(y_true, y_pred, squared=True)
print(f"MSE Score: {mse_score}")

rmse_score = mean_squared_error(y_true, y_pred, squared=False)
print(f"RMSE Score: {rmse_score}")

mae_score = mean_absolute_error(y_true, y_pred)
print(f"MAE Score: {mae_score}")

score_dict = {
    "R2 Score": r2_metric,
    "MSE Score": mse_score,
    "RMSE Score": rmse_score,
    "MAE Score": mae_score,
}

R2 score: -1.3166329749700343
MSE Score: 54630.78082984083
RMSE Score: 233.73228452620924
MAE Score: 186.5319760131836


/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/Aarav/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [46]:
def mgn(train_dir, val_dir, test_dir, callback_intervals, batch_size, epochs, iterations):
    #     print(
    #     "\n** This is the AttentiveFP Regressor, for retention-time prediction, RMSE optimized **\n"
    # )
    
    ############################################################################
    # Reading featurized, and split data
    ############################################################################
    
    train_dataset = dc.data.DiskDataset(data_dir=args.train_dir)
    valid_dataset = dc.data.DiskDataset(data_dir=args.val_dir)
    test_dataset = dc.data.DiskDataset(data_dir=args.test_dir)

    
    ############################################################################
    # Defining Search Space
    ############################################################################
    
    search_space = {
        ##* AttentiveFP
        "num_layers": hp.randint("num_layers", high=6, low=1),
        "graph_feat_size": hp.randint("graph_feat_size", high=300, low=30),
        "dropout": hp.uniform("dropout", high=0.5, low=0.0),
        "learning_rate": hp.loguniform(
            "learning_rate", high=(np.log(0.1)), low=(np.log(0.0001))
        ),
        "weight_decay_penalty": hp.loguniform(
            "weight_decay_penalty", high=(np.log(0.01)), low=(np.log(0.00001))
        ),
    }
    
    ############################################################################
    # Model training
    ############################################################################
    
    metric = dc.metrics.Metric(dc.metrics.mean_squared_error)
    epochs = args.epochs
    batch_size = args.batchsize
    validation_interval = args.callback_intervals
    n_tasks = len(train_dataset.tasks)
    
    
    def fm(search_args):
        model = dc.models.AttentiveFPModel(
            n_tasks=n_tasks,
            mode="regression",
            model_dir=model_check_point,
            batch_size=batch_size,
            ##* AttentiveFP Search Arguments
            num_layers=search_args["num_layers"],
            graph_feat_size=search_args["graph_feat_size"],
            dropout=search_args["dropout"],
            learning_rate=search_args["learning_rate"],
            weight_decay=search_args["weight_decay_penalty"],
        )
    
        # validation callback that saves the best checkpoint, i.e the one with the maximum score.
        callbacks = dc.models.ValidationCallback(
            valid_dataset,
            validation_interval,
            [metric],
            save_dir=model_check_point,
            # transformers=transformers,
            save_on_minimum=True,
        )
    
        model.fit(train_dataset, nb_epoch=epochs, callbacks=callbacks)
    
        # restoring the best checkpoint and passing the negative of its validation score to be minimized.
        model.restore(model_dir=model_check_point)
        valid_score = model.evaluate(valid_dataset, [metric])
        return valid_score["mean_squared_error"]
    
    
    ############################################################################
    # Hyperparameter optimization trials
    ############################################################################
    
    trials = Trials()
    best = fmin(
        fm,
        space=search_space,
        algo=tpe.suggest,
        max_evals=args.iterations,
        trials=trials,
    )
    
    
    ##* outputting trial results
    
    print("Best Parameters: {}".format(best))
    
    
    
    ############################################################################
    # Evaluating on test data
    ############################################################################
    n_tasks = len(test_dataset.tasks)
    model = dc.models.AttentiveFPModel(
        n_tasks=n_tasks,
        mode="regression",
        model_dir=model_check_point,
        batch_size=batch_size,
        ##* AttentiveFP Search Arguments
        num_layers=best["num_layers"],
        graph_feat_size=best["graph_feat_size"],
        dropout=best["dropout"],
        learning_rate=best["learning_rate"],
        weight_decay=best["weight_decay_penalty"],
    )
    
    
    print("*** Training model based on best parameters ***")
    model.fit(train_dataset, nb_epoch=epochs)
    
    
    print("making predictions on test...")
    preds = model.predict(test_dataset)
    preds = np.squeeze(preds)
    
    df = pd.DataFrame({"actual_RT": test_dataset.y, "pred_RT": preds})
    y_true = df["actual_RT"]
    y_pred = df["pred_RT"]
    
    
    r2_score = r2_score(y_true, y_pred)
    print(f"R2 score: {r2_score}")
    
    mse_score = mean_squared_error(y_true, y_pred, squared=True)
    print(f"MSE Score: {mse_score}")
    
    rmse_score = mean_squared_error(y_true, y_pred, squared=False)
    print(f"RMSE Score: {rmse_score}")
    
    mae_score = mean_absolute_error(y_true, y_pred)
    print(f"MAE Score: {mae_score}")
    
    
    score_dict = {
        "R2 Score": r2_score,
        "MSE Score": mse_score,
        "RMSE Score": rmse_score,
        "MAE Score": mae_score,
    }

    
    print("\n*** Hyperparameter Optimization is done ***\n")

In [ ]:
# def run_splits():
#     for i in range(5):
#         print(f"Iteration {i}")

#         epochs = 200
#         iterations = 100
#         batch = 30
#         callback = 1000

#         train_dir = f'features/rdkit_csv/cv_splits/labels_train_{i}_split.csv'
        
#         best_params = mgn(

